# 00 — EDA (clean)

This notebook is **EDA only**. No modeling, no feature leakage.

Checklist:
- Load `train.csv`/`test.csv` from `data/raw/`.
- Overview of columns, types, missingness.
- Target distribution, basic pivots.
- Suspicious data patterns or potential leakage?
